A Simple Digit Recognizer from the infamous MNIST dataset. This is actually called the "Hello World!" for Neural Networks. The project is taking help from a kaggle note book adn two you tube videos.

1. Building a neural network FROM SCRATCH (no Tensorflow/Pytorch, just numpy & math) by Samson Zhang - https://www.youtube.com/watch?v=w8yWXqWQYmU
2. Neural networks Series by 3Blue1Brown (First 4 videos are good enough for the project) - https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi


In [ ]:
#Importing the needed libraries and connecting to the dataset
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data  =pd.read_csv("train.csv")